## Carrying out multiple sequence alignments for all promoter seqeunces present across the strain collection

### Text chunk to check output

In [28]:
from Bio.Align.Applications import MuscleCommandline
muscle_cline = MuscleCommandline(input="/home/breena/Documents/Project/consensus_test/SigB_Plmo0995.fasta")
print(muscle_cline)

stdout, stderr = muscle_cline()
from io import StringIO
from Bio import AlignIO
align = AlignIO.read(StringIO(stdout), "fasta")
print(align)
count = SeqIO.write(align, "lmo0995.aln", "fasta")
filename = "lmo0995.aln"
format = "fasta"

muscle -in /home/breena/Documents/Project/consensus_test/SigB_Plmo0995.fasta
Alignment with 168 rows and 55 columns
GATACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- MQ140032_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1003_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1006_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1013_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1095_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1107_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1109_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1123_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1147_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1203_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1313_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGAATCT...--- 1371_SigB_Plmo0995
--TACTTTTTGTTACTTAACTGTTGGATAAGCGTTACAGA

### Wrote a bash script (.sh) file where all the input/output file paths got written and then this file could be ran via bash in terminal

In [ ]:
from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align import AlignInfo
from io import StringIO
from os import listdir
from os.path import isfile, join

output = open("/home/breena/Documents/Project/muscle_clwstrict.sh", "w+")
#path="/home/breena/Documents/Project/ordered_seqs/"
path='/home/breena/Documents/Project/cured_promoter_seqs_unordered/'
data = [f for f in listdir(path) if isfile(join(path, f))]
for file in data:
    #print(file)
    gene_name = file.replace("SigB_P", "").replace(".fasta", "")
    print(gene_name)
    cline = MuscleCommandline(input=path + file, out="/home/breena/Documents/Project/align/" + gene_name + ".aln")
    print(cline)
    output.write(str(cline) + ' -clwstrict' + '\n')
output.close()


### Visualising the sequence alignments

In [3]:
pip install jupyter biopython panel bokeh

     |████████████████████████████████| 9.9 MB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 10.7 MB 19.6 MB/s eta 0:00:01    |███▎                            | 1.1 MB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 5.2 MB/s  eta 0:00:01
     |████████████████████████████████| 97 kB 8.4 MB/s  eta 0:00:01
  Created wheel for bokeh: filename=bokeh-2.3.3-py3-none-any.whl size=11342778 sha256=0087c7f746767a9c719aa19c088ca15d8c88147e5e2bf96d22517b569924213f
  Stored in directory: /home/breena/.cache/pip/wheels/3d/1f/d4/a4ed6f21d242a5e2f4659b0c94d5b590aaf900773103406a97
Successfully built bokeh
  Attempting uninstall: bokeh
    Found existing installation: bokeh 2.2.3
    Uninstalling bokeh-2.2.3:
      Successfully uninstalled bokeh-2.2.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os, io, random
import string
import numpy as np

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

import panel as pn
import panel.widgets as pnw
pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot

# use AlignIO.read() function to read into file (give full path)
aln = AlignIO.read('/home/breena/Documents/Project/lmo0995.aln','clustal')
def get_colors(seqs):
    """make colors for bases in sequence"""
    text = [i for s in list(seqs) for i in s]
    clrs =  {'A':'red','T':'green','G':'orange','C':'blue','-':'white'}
    colors = [clrs[i] for i in text]
    return colors
def view_alignment(aln, fontsize="9pt", plot_width=800):
    """Bokeh sequence alignment view"""

    #make sequence and id lists from the aln object
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]    
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)    
    N = len(seqs[0])
    S = len(seqs)    
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')
    if N>100:
        viewlen=100
    else:
        viewlen=N
    #view_range is for the close up view
    view_range = (0,viewlen)
    tools="xpan, xwheel_zoom, reset, save"

    #entire sequence view (no text, with zoom)
    p = figure(title=None, plot_width= plot_width, plot_height=50,
               x_range=x_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False  

    #sequence text view with ability to scroll along x axis
    p1 = figure(title=None, plot_width=plot_width, plot_height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)          
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0

    p = gridplot([[p],[p1]], toolbar_location='below')
    return p

p = view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p)



Bokeh(Column)